<a href="https://colab.research.google.com/github/ssvadla/Legal_Text_Classification/blob/main/PseudoLabeling_LGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from google.colab import drive
import numpy as np
import statistics
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from collections import Counter
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.utils import resample
from sklearn.metrics import classification_report
import math
from nltk.corpus import stopwords
drive.mount('/content/drive')

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train1 = pd.read_csv('/content/drive/My Drive/Research/train_data1.csv')
train2 = pd.read_csv('/content/drive/My Drive/Research/train_data2.csv')
train3 = pd.read_csv('/content/drive/My Drive/Research/train_data3.csv')
train4 = pd.read_csv('/content/drive/My Drive/Research/train_data4.csv')
train5 = pd.read_csv('/content/drive/My Drive/Research/train_data5.csv')
train6 = pd.read_csv('/content/drive/My Drive/Research/train_data6.csv')
train7 = pd.read_csv('/content/drive/My Drive/Research/train_data7.csv')
train8 = pd.read_csv('/content/drive/My Drive/Research/train_data8.csv')
train9 = pd.read_csv('/content/drive/My Drive/Research/train_data9.csv')
train10 = pd.read_csv('/content/drive/My Drive/Research/train_data10.csv')
train_highKappa = pd.read_csv('/content/drive/My Drive/Research/train_data_highkappa.csv')
    #train1.head()
train = train1
train_list = [train2,train3,train4,train5,train6,train7,train8,train9,train10,train_highKappa]
for i in train_list:
  #print(i)
  train = train.append(i)
  
train.sort_values("Sentence", inplace = True)
print(len(train))

train = train.drop_duplicates(subset ="Sentence")

train['Target'].unique()


train['Target']=train['Target'].replace(['Others'],'Invalid')
train['Target'].unique()



#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

#cleaning data
def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)
X_tfidf = tfidf_vect.fit_transform(train['Sentence'])

#reading the test data
test = pd.read_csv(r'/content/drive/My Drive/Research/test_data.csv')


37711
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
(4416, 7374)


In [ ]:
X_train, x_val, Y_train, y_val = train_test_split(X_tfidf,train['Target'],test_size=0.20,random_state=42)
#Initialize the classifier
classifier = lgb.LGBMClassifier()
classifier.fit(X_train, Y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
#Unlabel data preprocessing
unlabel = pd.read_csv(r'/content/drive/My Drive/Research/Unlabeled_data.csv')
#unlabel.head()

del unlabel['Complete']
del unlabel['Unnamed: 0']

unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
unlabel['text'] = unlabel['text'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x for x in x.split() if x not in words))


from textblob import TextBlob
from textblob import Word
nltk.download('wordnet')
nltk.download('punkt')
unlabel['text'] = unlabel['text'].apply(lambda x: TextBlob(x).words)
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))



def index_reset(unlabel_2):
  unlabel_2.reset_index(inplace=True)
  del unlabel_2['index']
  return unlabel_2



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
unlabel_size_list = [500,1000]
#unlabel_size = 500 #comment
Threshold = 0.9 # comment

In [ ]:
train = train.rename(columns={'Sentence':'text'})

In [ ]:


record_unlabel_size_list = []
record_concat_train_size_list = []
record_upsample_size = []
record_cross_f1 = [] 
record_cross_accuracy = []
record_SS_CR = []
record_SS_F1 = []

#looping through the unlabel data
for unlabel_size in unlabel_size_list:

  record_unlabel_size_list.append(unlabel_size)
  unlabel_1 = unlabel.loc[:unlabel_size]
  unlabel_1 = index_reset(unlabel_1)
  tfidf_vect = TfidfVectorizer(analyzer = clean_text)
  X_tfidf = tfidf_vect.fit_transform(train['text'])
  x_un1 = tfidf_vect.transform(unlabel_1['text'])
  print(x_un1.shape)
  pred_unlabel_1 = classifier.predict_proba(x_un1)
  pred_unlabel_1
  x_un1.shape
  import numpy as np
  pos=[]
  large=[]
  ind = []
  i=0
  for j in pred_unlabel_1:
    if max(j)> Threshold:
      ind.append(np.argmax(j))
      large.append(max(j))
      pos.append(i)
    i+=1
  unlabel_1 = unlabel_1.loc[pos,:]
  train_data_size = len(unlabel_1)
  class_x_un1 = tfidf_vect.transform(unlabel_1['text'])
  class_x_un1.shape
  class_pred_unlabel_1 = classifier.predict(class_x_un1)
  class_pred_unlabel_1
  unlabel_1['Target']=class_pred_unlabel_1
  train = train.rename(columns={'Sentence':'text'})
  frame_1 = [train,unlabel_1]
  train_1 = pd.concat(frame_1)
  record_concat_train_size_list.append(len(train_1))
  target_values = np.unique(train_1['Target'].values)
  data_list = []
  data_length_list = []
  for i in target_values:
    df_k = train_1[train_1['Target']==i]
    data_list.append(df_k)
    data_length_list.append(len(df_k))
  maximum_data = max(data_length_list)
  ratio = math.floor(( 4 / 6 )* maximum_data)
  loop_count = 0
  train_upsampled = []
  for i in data_length_list:
    if i < ratio:
      df_upsampled = resample(data_list[loop_count],replace=True,n_samples=ratio,random_state=123)
    else:
      df_upsampled = resample(data_list[loop_count],replace=True,n_samples=i,random_state=123)
    train_upsampled.append(df_upsampled)
    loop_count = loop_count + 1
  df_res = pd.concat(train_upsampled)
  record_upsample_size.append(len(df_res))
  tfidf_vect = TfidfVectorizer(analyzer = clean_text)
  X_tfidf = tfidf_vect.fit_transform(df_res['text'])
  X_tfidf_df=pd.DataFrame(X_tfidf.toarray())
  X_tfidf_df.columns=tfidf_vect.get_feature_names()
  X_train, x_val, Y_train, y_val = train_test_split(X_tfidf,df_res['Target'],test_size=0.20,random_state=42)
  classifier_1 = lgb.LGBMClassifier()
  classifier_1.fit(X_train, Y_train)
  y_pred = classifier_1.predict(x_val)
  Accuracy_score = accuracy_score(y_val, y_pred)
  cv = KFold(n_splits=5, random_state=1, shuffle=True)
  scores_f1 = cross_val_score(classifier_1,x_val ,y_val, scoring = 'f1_weighted', cv=cv, n_jobs=-1).mean()
  record_cross_f1.append(scores_f1)
  scores_acc = cross_val_score(classifier_1,x_val ,y_val, scoring = 'accuracy', cv=cv, n_jobs=-1).mean()
  record_cross_accuracy.append(scores_acc)
  test['Target']=test['Target'].replace(['Others'],'Invalid')
  test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
  test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
  words = stopwords.words('english')
  test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))
  t_p = tfidf_vect.transform(test['Sentence'])
  test_pred = classifier_1.predict(t_p)
  classification_report_semi_supervised = classification_report(test['Target'],test_pred, digits=4)
  record_SS_CR.append(classification_report_semi_supervised)
  f1_Score_metric = f1_score(test['Target'],test_pred,average='weighted')
  record_SS_F1.append(f1_Score_metric)
    



(501, 7374)
57
678
678
@ 1526
235
235
@ 1526
2289
2289
@ 2289
667
667
@ 1526
269
269
@ 1526
335
335
@ 1526
(1001, 7374)
139
678
678
@ 1571
240
240
@ 1571
2357
2357
@ 2357
674
674
@ 1571
270
270
@ 1571
336
336
@ 1571


In [ ]:

#write the output to a text file
with open(r'/content/drive/My Drive/Results/LGBM_UL_Size_Test_cross_01/Result.txt', 'a') as writefile:
  writefile.write(" Semi UL test experiment : ")
  writefile.write("\n")
  writefile.write(" record_unlabel_size_list ")
  writefile.write("\n")
  writefile.write(str(record_unlabel_size_list))
  writefile.write("\n")
  writefile.write(" record_concat_train_size_list ")
  writefile.write("\n")
  writefile.write(str(record_concat_train_size_list))
  writefile.write("\n")
  writefile.write(" record_upsample_size ")
  writefile.write("\n")
  writefile.write(str(record_upsample_size))
  writefile.write("\n")
  writefile.write(" record_cross_f1 ")
  writefile.write("\n")
  writefile.write(str(record_cross_f1))
  writefile.write("\n")
  writefile.write(" record_cross_accuracy")
  writefile.write("\n")
  writefile.write(str(record_cross_accuracy))
  writefile.write("\n")
  writefile.write(" record_SS_CR ")
  writefile.write("\n")
  writefile.write(str(record_SS_CR))
  writefile.write("\n")
  writefile.write(" record_SS_F1")
  writefile.write("\n")
  writefile.write(str(record_SS_F1))
  writefile.write("\n")
